In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import json
import re
import warnings
from IPython.display import clear_output
import string

In [2]:
def iterator(L):
	a, b = itertools.tee(L)
	next(b, None)
	return list(zip(a,b))

In [3]:
f = open("subwaygraph.json",'r')
G = nx.node_link_graph(json.load(f), directed=False, multigraph=True, edges='edges')

In [4]:
stations = pd.read_csv("MTA_Subway_Stations.csv")

stations = stations[['GTFS Stop ID','Complex ID','Stop Name','Daytime Routes']]
stations.set_index('GTFS Stop ID',inplace=True)
stations.rename_axis("id", axis=0, inplace=True)
stations = stations[stations['Daytime Routes'] != 'SIR']
stations.rename({"Daytime Routes":"lines", "Stop Name": "name"}, axis='columns', inplace=True)
stations.sort_index(inplace=True)
stations['lines'] = [l.split() for l in stations['lines']]


In [5]:

		
class Subway:
	def __init__(this, G, station_names=pd.DataFrame):
		this.G = G
		this.stations = station_names
		
		this.route_count = 0
		this.routes = {}

		
	def find_stop(this, name, line_spec=None):
		stations = this.stations
		opts = stations[stations.name.str.contains(name, case=False)]
		if len(opts) == 0: 
			raise RuntimeError("No stations found")
		elif len(opts) == 1:
			return list(opts.index)
		elif len(opts)>1:
			if line_spec:
				line_spec = str(line_spec)
				stopmatch = opts[opts.lines.map(lambda x: line_spec.capitalize() in x)]
				return list(stopmatch.index)
			else:
				print("Multiple stations with name {} found. Select subway line from below".format(name))
				print(opts.head())
				line = str(input("line"))
				if line == 'default': start_id = opts[1]
				else:
					start_id = opts[opts.lines.map(lambda x: line.capitalize() in x)]
					if len(start_id) == 0: 
						warnings.warn("Line not found, defaulting to first station")
						return list(opts.index)
					return list(start_id.index)
		else: raise RuntimeError("how did you manage to mess this up either it exists or it doesnt")
				
	def new_route(this, start, stop, start_line=None, stop_line=None, id=False):
		if not id:
			start = this.find_stop(start, line_spec=start_line)[0]
			stop = this.find_stop(stop, line_spec=stop_line)[0]
			clear_output()
			
		this.route_count += 1
		rname = "route{}".format(this.route_count)
		this.routes[rname] = (start, stop)

	def new_routes(this, lst, id=False):
		for i,j in lst:
			this.new_route(i,j, id=id)

	def remove_route(this, route):
		try: del this.routes[route]
		except KeyError: pass


	def clear_routes(this):
		this.routes = 0
		this.route_count = 0

In [6]:
rng = np.random.default_rng(112)
stop_pairs_id = rng.choice(list(G.nodes()),size=(2,2), replace=False).tolist()
stop_pairs_name = [[stations.loc[i]['name'] for i in k] for k in stop_pairs_id]
stop_pairs_name

[['New Lots Av', 'Jackson Av'], ['East 105 St', 'Cypress Hills']]

In [7]:
nyc = Subway(G, stations)

In [8]:
nyc.new_route("fulton", "broadway j", "z", "z")

In [9]:
nyc.routes

{'route1': ('M22', 'J27')}

In [10]:
path

NameError: name 'path' is not defined

In [ ]:
path = nx.dijkstra_path(G, nyc.routes['route1'][0], nyc.routes['route1'][1], weight='time')
[[stations.loc[i]['name'], G.edges[i,j,0]['lines']] for i,j in iterator(path)]

[['Fulton St', ['Transfer']],
 ['Fulton St', ['A', 'C']],
 ['High St', ['A', 'C']],
 ['Jay St-MetroTech', ['A', 'C']],
 ['Hoyt-Schermerhorn Sts', ['A', 'C']],
 ['Nostrand Av', ['A', 'C']],
 ['Utica Av', ['A', 'C']],
 ['Broadway Junction', ['Transfer']]]